# Flow with symlinks

When there is a cross flow share scenario in the flow folder, although using symlink can facilitate development, it has the following limitations. It is recommended to use **additional include**.

1. For Windows user, need to Administrator to create symbolic.
2. For Windows user, directly copy the folder with symlinks, it will deep copy the contents to the location.
3. Need to update the git config to support symlinks.


**Requirements**:
-  For Windows user, please creating user permission to [create symbolic links](https://learn.microsoft.com/en-us/windows/security/threat-protection/security-policy-settings/create-symbolic-links).
    1. Open your `Local Security Policy`
    2. Find `Local Policies` -> `User Rights Assignment` -> `Create symbolic links`
    3. Add you user name to this policy then reboot the compute.

**Attention**:
- For git operations, need to set `git config core.symlinks true`

## 0. Create symbolic links in the flow

In [ ]:
! python ../../flows/standard/flow-with-symlinks/create_symlinks.py

## 1. Test flow in local

Execute promptflow CLI to test the flow in local.

In [ ]:
! pf flow test --flow ../../flows/standard/flow-with-symlinks

## 2. Create a new batch run in local

### 2.1 Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

In this notebook, we will use flow `flow-with-symlinks` which uses connection `azure_open_ai_connection` inside, we need to set up the connection if we haven't added it before. After created, it's stored in local db and can be used in any flow.

In [ ]:
import promptflow as pf
from promptflow.connections import AzureOpenAIConnection

# client can help manage your runs and connections.
client = pf.PFClient()

try:
    conn_name = "azure_open_ai_connection"
    conn = client.connections.get(name=conn_name)
    print("using existing connection")
except:
    connection = AzureOpenAIConnection(
        name=conn_name,
        api_key="<test_key>",
        api_base="<test_base>",
        api_type="azure",
        api_version="<test_version>",
    )

    conn = client.connections.create_or_update(connection)
    print("successfully created connection")

print(conn)

### 2.2 Create a new batch run

In [ ]:
flow = "../../flows/standard/flow-with-symlinks" # path to the flow directory
data = "../../flows/standard/flow-with-symlinks/data.jsonl" # path to the data file

# create batch run with default variant
base_run = pf.run(
    flow=flow,
    data=data
)

pf.stream(base_run)

In [ ]:
details = pf.get_details(base_run)
details.head(10)

## 3. Create a new branch run in azure

### 3.1 Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.


In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
# azure version promptflow apis
import promptflow.azure as pf

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

### 3.2 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# configure global setting pointing to workpsace ml_client
pf.configure(ml_client)
runtime = "demo-mir" # TODO remove this after serverless ready

### 3.3 Create a new batch run

In [ ]:
flow = "../../flows/standard/flow-with-symlinks" # path to the flow directory
data = "../../flows/standard/flow-with-symlinks/data.jsonl" # path to the data file

# create batch run with default variant
base_run = pf.run(
    flow=".",
    data="data.jsonl",
    runtime=runtime,
)

pf.stream(base_run)

In [ ]:
details = pf.get_details(base_run)
details.head(10)